<h1 align = "center">Grouo2-Week4-Reporting</h1>
<h4 align = "right">Members:杨开创、王媛、赵心研、樊宇鑫</h4>
<h4 align = "right">Reporter:杨开创</h4>

## 1. Modules Dedign





<li>data.py</li>

<li>model.py</li>

<li>util.py</li>

<li>extract_summary_finetune_train.py</li>

<li>preformance_measures.py</li>

<li>click: cmd_run.py</li>

<li>extract_summary_news.py</li>



In [1]:
import click
import os
import torch
import dill
import torch.nn as nn
from torch.utils.data import DataLoader

from extract_summary_finetune.data import SummaryData,SummaryDataset
from extract_summary_finetune.model import lstm,transformer
from extract_summary_finetune.util import set_seed
from extract_summary_finetune.extract_summary_finetune_train import train_model
from extract_summary_finetune.performance_measures import calc_auc_score
from extract_summary_finetune.extract_summary_from_news import ExtratSummary


## 2. Model Selection


<li> LSTM </li>
<li> Transformer(Encoder) </li>

model_settings:

<li> input_size : 1024 for wwm_bert and swufe_bert ,768 for cn_bert </li>
<li> time_step : 512 </li>
<li> epoch =  5 </li>
<li> learinf_rate = 0.0003 </li>
<li> model: LSTM(1,2,3,4) , Transformer(1,2,3,4) </li>
<li> bert : swufe_bert , wwm_bert , cn_bert </li>
<li> loss: BCEloss , Weighted_BCEloss </li>
type = 8 * 3 * 2 = 48


In [2]:

    input_size = 1024 
     
    time_step = 512
    
    epoch =  5 
    
    learinf_rate = 0.0003 
    
    loss = "w_BCEloss"
    
    

weighted_loss

In [3]:
import torch
import torch.nn as nn
import math
import numpy as np

def loss_weighted(pred,label):
    if (pred.ndim == 3)+(label.ndim == 3)==2:
        weight = []
        for i in range(label.shape[0]):
            sumamary_sign_postion  = -1
            distance_score = [0 for i in range(label.shape[1])]
            for j in range (label.shape[1]):
                distance_score[j] = math.log((j-sumamary_sign_postion)+1)
                if (j>0) & (label[i][j-1][0].item()==1) :
                    distance_score[j] = distance_score[j-1]
                if label[i][j][0].item()==1:
                    sumamary_sign_postion = j
            row_weight = [[label.shape[1]*item/sum(distance_score)] for item in distance_score]
            weight.append(row_weight)
        return torch.tensor(weight,dtype=float)
    else:
        return "InputError: true_label and pred_label must be .ndim=3 "
    
    if is_weighted:
        loss = loss_func(weight=loss_weighted(pred,label).float())(pred, label)
    else:
        loss = loss_func()(pred, label)


In [4]:

    model_setting_parameters = {
    'lstm1':{'input_size':input_size, 'hidden_size':64, 'num_layers':1, 'dropout':0},
    'lstm2':{'input_size':input_size, 'hidden_size':128, 'num_layers':1, 'dropout':0},
    'lstm3':{'input_size':input_size, 'hidden_size':64, 'num_layers':2, 'dropout':0.05},
    'lstm4':{'input_size':input_size, 'hidden_size':128, 'num_layers':2, 'dropout':0.05},
    'transformer1':{'d_model':input_size, 'src_len':time_step,'dim_feedforward':128, 'nhead':4, 'num_encoder_layers':1, 'activation':"relu",'dropout':0.05},
    'transformer2':{'d_model':input_size, 'src_len':time_step,'dim_feedforward':256, 'nhead':4, 'num_encoder_layers':1, 'activation':"relu",'dropout':0.05},
    'transformer3':{'d_model':input_size, 'src_len':time_step,'dim_feedforward':128, 'nhead':4, 'num_encoder_layers':2, 'activation':"relu",'dropout':0.05},
    'transformer4':{'d_model':input_size, 'src_len':time_step,'dim_feedforward':256, 'nhead':4, 'num_encoder_layers':2, 'activation':"relu",'dropout':0.05}
    }

## 3. Results
<li>log</li>
<li>tensorboard</li>

(lstm1, trainsformer1)*(swufe_bert , wwm_bert , cn_bert)*(BCEloss , Weighted_BCEloss) = 12

loss & auc

In [12]:
%load_ext tensorboard
%tensorboard --logdir runs/train

In [5]:
from extract_summary_finetune.display_by_tensorboard import MyTensorboard

tb_loss_auc = MyTensorboard("/home/yuxin.fan/tensorboard_display","/home/yuxin.fan/result/")
tb_loss_auc.read_file('summary_lstm1_cn_bert_BCEloss.dil')
tb_loss_auc.step_loss()
tb_loss_auc.epoch_metric()
tb_loss_auc.tensorboard_close()

hparams

In [6]:
tb_hparams = MyTensorboard("/home/yuxin.fan/tensorboard_display","/home/yuxin.fan/result/")
tb_hparams.file_hparams(lr=0.0005, batch_size=50)
tb_hparams.tensorboard_close()

cn/wwm/swufe

In [7]:
tb_3_models = MyTensorboard("/home/yuxin.fan/tensorboard_display","/home/yuxin.fan/result/")
tb_3_models.models_loss_auc()
tb_3_models.tensorboard_close()

![jupyter](/home/yuxin.fan/extract_summary_finetune/note_book/picture/sendpix0.jpg)
![jupyter](/home/yuxin.fan/extract_summary_finetune/note_book/picture/sendpix1.jpg)
![jupyter](/home/yuxin.fan/extract_summary_finetune/note_book/picture/sendpix2.jpg)

![jupyter](/home/yuxin.fan/extract_summary_finetune/note_book/picture/sendpix3.jpg)
![jupyter](/home/yuxin.fan/extract_summary_finetune/note_book/picture/sendpix4.jpg)
![jupyter](/home/yuxin.fan/extract_summary_finetune/note_book/picture/sendpix5.jpg)
![jupyter](/home/yuxin.fan/extract_summary_finetune/note_book/picture/sendpix6.jpg)

![jupyter](/home/yuxin.fan/extract_summary_finetune/note_book/picture/sendpix7.jpg)

## 4. Summary Example

In [8]:
from extract_summary_finetune.extract_summary_from_news import ExtratSummary
s1 = ExtratSummary(network="lstm1",bert="cn_bert",loss="BCEloss",data_path='/home/yuxin.fan/feature_from_cn_bert.npy').results_dataframe
s2 = ExtratSummary(network="lstm1",bert="wwm_bert",loss="BCEloss",data_path='/home/yuxin.fan/feature_from_wwm_bert.npy').results_dataframe
s3 = ExtratSummary(network="lstm1",bert="swufe_bert",loss="BCEloss").results_dataframe
print('summary:')
print(f'cn:{s1["summary"][0]}')
print(f'wwm:{s2["summary"][0]}')![jupyter](/home/yuxin.fan/extract_summary_finetune/note_book/picture/sendpix7.jpg)
print(f'cn:{s1["summary_modified"][0]}')
print(f'wwm:{s2["summary_modified"][0]}')
print(f'swufe:{s3["summary_modified"][0]}')
print('news:')
print({s1['content'][0]})

summary:
cn:[汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元
wwm:[汽车官获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版erormance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价度均为2.3万元。长续航
swufe:[汽车之车官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万续航版的ND里6
modified_summary:
cn:[汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元
wwm:[汽车官获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元。
swufe:[汽车之车我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元续航版的ND里6
news:
{'[汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元。长续航版的NEDC续航里程为660km，0-100km加速时间为3.8秒；Performance高性能版0-100km加速时间仅为2.5秒。目前特斯拉（中国）官网显示的Model S共有三款车型，分别为长续航版、Performance 高性能版和Plaid版，此次调价仅针对前面两款车型。值得一提的是，特斯拉Model S的上一次调价是在7月份，全系车型下调幅度为0.8万元

In [9]:
def modify_labels(self):
        news_to_sentences = self.divide_sentences()
        predict_label_modified = []
        for k in range(len(news_to_sentences)):
            sentences_length_list = [len(sentence) for sentence in news_to_sentences[k]]
            start_index = 0
            label_modified_list = []
            for i in range(len(sentences_length_list)):
                end_index = start_index + sentences_length_list[i]
                label_divided = list(self.predict_label[k][start_index:end_index])
                if sum(label_divided) > 0.5*len(label_divided):
                    label_modified = [True for item in label_divided]
                elif sum(label_divided) < 0.2*len(label_divided):
                    label_modified = [False for item in label_divided]
                else:
                    label_modified = label_divided
                label_modified_list.extend(label_modified)
                start_index = end_index
            predict_label_modified.append(np.array(label_modified_list))
        return np.array(predict_label_modified)
    

In [15]:
s1 = ExtratSummary(network="transformer1",bert="cn_bert",loss="BCEloss",data_path='/home/yuxin.fan/feature_from_cn_bert.npy').results_dataframe
s2 = ExtratSummary(network="transformer1",bert="wwm_bert",loss="BCEloss",data_path='/home/yuxin.fan/feature_from_wwm_bert.npy').results_dataframe
s3 = ExtratSummary(network="transformer1",bert="swufe_bert",loss="BCEloss").results_dataframe
print('summary:')
print(f'cn:{s1["summary"][0]}')
print(f'wwm:{s2["summary"][0]}')
print(f'swufe:{s3["summary"][0]}')
print('modified_summary:')
print(f'cn:{s1["summary_modified"][0]}')
print(f'wwm:{s2["summary_modified"][0]}')
print(f'swufe:{s3["summary_modified"][0]}')
print('news:')
print({s1['content'][2]})

summary:
cn:[汽车之家 新车上市 前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万长
wwm:新车上市前们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.9万，降价度均3航
swufe:斯拉（国）网针对在售o参数|询价)续航版、Performance高性能版的价格进了下调，调整后价为73.39万83.万度2.3D特化，动C
modified_summary:
cn:[汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元
wwm:新车上市前我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价度均3
swufe:特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元度2.3，
news:
{'在2018年的大牛行情过后，2019年的债市将如何演绎？多位基金经理在接受记者采访时表示，去年的快速上涨收窄了债市继续上扬的空间，尤其是利率债的机会明显减少，但整体牛市的格局不会变化，今年可能处于阶段性的“牛尾行情”。基于这一判断，许多基金经理将债市淘金的对象指向信用债，期望从“信用利差”中获取超额收益。适度下沉信用资质\\u3000\\u3000在基金经理眼里，利率债的强势上扬催生了去年的大牛市，今年信用债有可能成为债市新主角，为债基提供了结构性投资机会。她指出，历史上的债券行情，一般以利率债为开端，此后轮动到高、中、低等级信用债，最后以垃圾债收尾。建信中短债纯债基金的拟任基金经理李峰也表示，去年由于信用债违约潮的出现，利率债和信用债的利差不减反增，直到2018年四季度才有所好转。随着今年利率债的投资价值降低和宽货币效果的逐步显

In [11]:
s1 = ExtratSummary(network="lstm1",bert="cn_bert",loss="w_BCEloss",data_path='/home/yuxin.fan/feature_from_cn_bert.npy').results_dataframe
s2 = ExtratSummary(network="lstm1",bert="wwm_bert",loss="w_BCEloss",data_path='/home/yuxin.fan/feature_from_wwm_bert.npy').results_dataframe
s3 = ExtratSummary(network="lstm1",bert="swufe_bert",loss="w_BCEloss").results_dataframe
print('summary:')
print(f'cn:{s1["summary"][0]}')
print(f'wwm:{s2["summary"][0]}')
print(f'swufe:{s3["summary"][0]}')
print('modified_summary:')
print(f'cn:{s1["summary_modified"][0]}')
print(f'wwm:{s2["summary_modified"][0]}')
print(f'swufe:{s3["summary_modified"][0]}')
print('news:')
print({s1['content'][0]})

summary:
cn:[汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万
wwm:[汽车，从官方获悉，特斯拉（中国）官网针对在售的Mode S数|询价)ance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降度均为2.3万元
swufe:[汽车之新车 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价度均为2.3万
modified_summary:
cn:[汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元
wwm:[汽车，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元
swufe:[汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元
news:
{'[汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元。长续航版的NEDC续航里程为660km，0-100km加速时间为3.8秒；Performance高性能版0-100km加速时间仅为2.5秒。目前特斯拉（中国）官网显示的Model S共有三款车型，分别为长续航版、Performance 高性能版和Plaid版，此次调价仅针对前面两款车型。值得一提的是，特斯拉Model S的上一次调价是在7月份，全系车型下调幅度为0.8万元。今年5月份的调价幅

In [11]:
s1 = ExtratSummary(network="transformer1",bert="cn_bert",loss="w_BCEloss",data_path='/home/yuxin.fan/feature_from_cn_bert.npy').results_dataframe
s2 = ExtratSummary(network="transformer1",bert="wwm_bert",loss="w_BCEloss",data_path='/home/yuxin.fan/feature_from_wwm_bert.npy').results_dataframe
s3 = ExtratSummary(network="transformer1",bert="swufe_bert",loss="w_BCEloss").results_dataframe
print('summary:')
print(f'cn:{s1["summary"][0]}')
print(f'wwm:{s2["summary"][0]}')
print(f'swufe:{s3["summary"][0]}')
print('modified_summary:')
print(f'cn:{s1["summary_modified"][0]}')
print(f'wwm:{s2["summary_modified"][0]}')
print(f'swufe:{s3["summary_modified"][0]}')
print('news:')
print({s1['content'][0]})

summary:
cn:车新车上市，们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价均为2.
wwm:新车上市前从官悉，特斯拉（中国）官网针对在售的Modl (参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.9价均航
swufe:拉（针对在售o参数|询价)续航版Performanc高性能版的价格进了下调，整后价为73.39万、23
modified_summary:
cn:车新车上市，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元
wwm:新车上市前从官悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元
swufe:特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元
news:
{'[汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元。长续航版的NEDC续航里程为660km，0-100km加速时间为3.8秒；Performance高性能版0-100km加速时间仅为2.5秒。目前特斯拉（中国）官网显示的Model S共有三款车型，分别为长续航版、Performance 高性能版和Plaid版，此次调价仅针对前面两款车型。值得一提的是，特斯拉Model S的上一次调价是在7月份，全系车型下调幅度为0.8万元。今年5月份的调价幅度更大，特斯拉Model S全系车型售价下调了2.9万元。特斯拉Model S的整体造型设计并没有变化，动力同样没有发生改动，长续航版和Performance高性能版均搭载双电机，总最大功率分别为487千瓦、577千瓦，0-100km/h加速时间分别

## 5. Problems and Questions

<li>1.transformer模型中的FFN层为什么要把每个字的维度扩大再还原。</li>
<li>2.加权loss的要怎么合理的设置，可以通过哪些有据可依的加权方法对于关注部分给予更高的权重。</li>
<li>3.由于本次语料规律性很强，比如来自某个网站的企业公告，自己在标注时就会先入为主的只看重点关注的位置。而本次语料库有大量格式相似的语料，是否可以利用python一些优秀的中文语言处理库，先对语料进行特征工程然后分类然后对loss 权重函数按类进行编写。</li>
